In [1]:
import os
import zipfile
import csv

import requests

def get_data():
    return csv.DictReader((x for x in open("data/FacAbr2020_f.csv",encoding = "utf-8")), delimiter=";",)

def get_ratings():
    return get_data()

def get_book_features():

    return get_data()

In [2]:
import json
from itertools import islice

ratings = get_data()

In [3]:
for line in islice(ratings, 2):
    print(json.dumps(line, indent=4))

{
    "User-ID": "\ufeff1000805531",
    "PRODUCT-ID": "106030",
    "PURCHASE": "1"
}
{
    "User-ID": "1000805531",
    "PRODUCT-ID": "135033",
    "PURCHASE": "1"
}


In [5]:
from lightfm.data import Dataset

dataset = Dataset()
dataset.fit((x['User-ID'] for x in get_ratings()),
            (x['PRODUCT-ID'] for x in get_ratings()))

In [6]:
num_users, num_items = dataset.interactions_shape()
print('Num users: {}, num_items {}.'.format(num_users, num_items))

Num users: 6608, num_items 2186.


In [6]:
(interactions, weights) = dataset.build_interactions(((x['User-ID'], x['ISBN'],float(x['Book-Rating']))
                                                      for x in get_ratings()))

print(repr(interactions))
print(repr(weights))

<1259x119 sparse matrix of type '<class 'numpy.int32'>'
	with 4487 stored elements in COOrdinate format>
<1259x119 sparse matrix of type '<class 'numpy.float32'>'
	with 4487 stored elements in COOrdinate format>


In [7]:
print(interactions)

  (0, 0)	1
  (1, 0)	1
  (2, 0)	1
  (3, 0)	1
  (4, 0)	1
  (5, 0)	1
  (6, 0)	1
  (7, 0)	1
  (8, 0)	1
  (9, 0)	1
  (10, 0)	1
  (11, 0)	1
  (12, 0)	1
  (13, 0)	1
  (14, 0)	1
  (15, 0)	1
  (16, 0)	1
  (17, 0)	1
  (18, 0)	1
  (19, 0)	1
  (20, 0)	1
  (21, 0)	1
  (22, 0)	1
  (23, 0)	1
  (24, 0)	1
  :	:
  (663, 117)	1
  (894, 117)	1
  (1253, 117)	1
  (151, 117)	1
  (454, 118)	1
  (82, 118)	1
  (1254, 118)	1
  (12, 118)	1
  (478, 118)	1
  (702, 118)	1
  (1255, 118)	1
  (173, 118)	1
  (1256, 118)	1
  (15, 118)	1
  (176, 118)	1
  (1257, 118)	1
  (19, 118)	1
  (490, 118)	1
  (86, 118)	1
  (73, 118)	1
  (28, 118)	1
  (1198, 118)	1
  (1258, 118)	1
  (1201, 118)	1
  (432, 118)	1


In [8]:
print(weights)

  (0, 0)	3.0
  (1, 0)	1.0
  (2, 0)	1.0
  (3, 0)	3.0
  (4, 0)	1.0
  (5, 0)	1.0
  (6, 0)	1.0
  (7, 0)	1.0
  (8, 0)	1.0
  (9, 0)	2.0
  (10, 0)	1.0
  (11, 0)	1.0
  (12, 0)	1.0
  (13, 0)	1.0
  (14, 0)	1.0
  (15, 0)	1.0
  (16, 0)	3.0
  (17, 0)	3.0
  (18, 0)	1.0
  (19, 0)	1.0
  (20, 0)	3.0
  (21, 0)	1.0
  (22, 0)	1.0
  (23, 0)	1.0
  (24, 0)	1.0
  :	:
  (663, 117)	2.0
  (894, 117)	1.0
  (1253, 117)	1.0
  (151, 117)	1.0
  (454, 118)	1.0
  (82, 118)	1.0
  (1254, 118)	1.0
  (12, 118)	1.0
  (478, 118)	1.0
  (702, 118)	1.0
  (1255, 118)	1.0
  (173, 118)	1.0
  (1256, 118)	1.0
  (15, 118)	3.0
  (176, 118)	1.0
  (1257, 118)	1.0
  (19, 118)	1.0
  (490, 118)	1.0
  (86, 118)	1.0
  (73, 118)	2.0
  (28, 118)	1.0
  (1198, 118)	1.0
  (1258, 118)	1.0
  (1201, 118)	1.0
  (432, 118)	2.0


In [9]:
from lightfm import LightFM

model = LightFM(loss='bpr')
model.fit(weights)

In [10]:
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import auc_score

train_precision = precision_at_k(model, weights, k=10).mean()

print('Precision: train %.2f' % (train_precision))

Precision: train 0.11
